# Install libs ที่เกี่ยวข้อง

In [4]:
pip install playwright html5lib requests beautifulsoup4 xlrd lxml papermill 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## ดึงข้อมูล diw : กรมอุตสาหกรรม ##

-------------------

# Extract data from diw

In [ ]:
from playwright.async_api import async_playwright
from datetime import datetime
import os

area_skip = []
province_skip = []
district_skip = []

parent_dir = 'diw_2'

async def main():
    start_time = datetime.now()
    broken_excel_file = [] # เพิ่ม list สำหรับเก็บอำเภอที่ดาวน์โหลดไฟล์ไม่ได้
    incomplete_downloads = {} # เพิ่ม dictionary สำหรับเก็บอำเภอที่ดาวน์โหลดไม่ครบ
    
    # กำหนดชื่อไฟล์ล็อก
    log_filename = f"log_diw_{datetime.now().strftime('%Y-%m-%d')}.txt"
    log_path = os.path.join('.', log_filename)
    
    # สร้างโฟลเดอร์สำหรับบันทึกไฟล์
    if not os.path.exists(f"{parent_dir}"):
        os.makedirs(f"{parent_dir}")
        
    async with async_playwright() as playwright:
        # Launch a browser
        browser = await playwright.chromium.launch(headless=False) # headless=False : ให้หน้า browser ขึ้นมาเวลารันเป็นเวลา slow_mo=500 : pop-up ขึ้นมาเป็นเวลา 0.5 secs
        
        # Create a new page
        page = await browser.new_page()

        target_url = 'https://userdb.diw.go.th/factoryPublic/tumbol.asp'

        await page.goto(target_url)
        
        # รอให้หน้าแรกโหลดเสร็จสมบูรณ์ก่อน
        await page.wait_for_load_state('domcontentloaded')

        area_arr = []

        area = await page.locator('//a').all()

        for a in area:
            area_arr.append(await a.text_content())

        # Loop area
        for a in area_arr:
            if a not in area_skip:
                
                print(f"กำลังดำเนินการในภูมิภาค: {a}") # แสดงผลการดำเนินการในระดับภูมิภาค
                await page.locator(f"//a[text()='{a}']").click()
                
                # สร้างโฟลเดอร์ย่อยสำหรับภูมิภาค
                area_dir = f"{parent_dir}/{a}"
                if not os.path.exists(area_dir):
                    os.makedirs(area_dir)
                    
                # รอให้หน้ารายชื่อจังหวัดโหลดเสร็จสมบูรณ์
                await page.wait_for_selector("//a[contains(text(),'จ.')]", timeout=10000)

                province_arr = []

                province = await page.locator("//a[contains(text(),'จ.')]").all()
                
                for p in province:
                    province_arr.append(await p.text_content())
                    
                print(province_arr)

                # Loop province
                for p in province_arr:
                    if p not in province_skip:
                                                
                        print(f"  -> กำลังดำเนินการในจังหวัด: {p}") # แสดงผลการดำเนินการในระดับจังหวัด
                        await page.locator(f"//a[text()='{p}']").click()
                                                
                        # รอให้หน้ารายชื่ออำเภอโหลดเสร็จสมบูรณ์
                        await page.wait_for_load_state('domcontentloaded')

                        # สร้างโฟลเดอร์ย่อยสำหรับจังหวัด
                        province_dir = f"{area_dir}/{p}"
                        if not os.path.exists(province_dir):
                            os.makedirs(province_dir)
                            
                        # รอให้หน้ารายชื่ออำเภอโหลดเสร็จสมบูรณ์
                        await page.wait_for_selector("//a[contains(text(),'อ.')]", timeout=10000)

                        district_arr = []

                        district = await page.locator("//a[contains(text(),'อ.')]").all()

                        for d in district:
                            district_arr.append(await d.text_content())
                            
                        # สร้าง set ของอำเภอที่ต้องดาวน์โหลด
                        expected_districts = {d for d in district_arr if 'กิ่งอำเภอ' not in d and d not in district_skip}

                        # Loop district
                        for d in district_arr:
                            if 'กิ่งอำเภอ' not in d and d not in district_skip:
                                print(f"    -> กำลังดำเนินการในอำเภอ: {d}") # แสดงผลการดำเนินการในระดับอำเภอ
                                await page.locator(f"//a[text()='{d}']").click()
                                
                                # รอให้ปุ่มดาวน์โหลดโหลดเสร็จสมบูรณ์
                                await page.wait_for_selector("//a[contains(text(),'download')]", timeout=10000)

                                try:
                                    # Start waiting for the download
                                    async with page.expect_download() as download_info:
                                        # Perform the action that initiates download
                                        await page.locator(f"//a[contains(text(),'download')]").click()

                                        download = await download_info.value

                                        # Wait for the download process to complete and save the downloaded file somewhere 
                                        
                                        # แก้ไขชื่อไฟล์ให้มีชื่อภูมิภาค, จังหวัด, และอำเภอ
                                        new_filename = f"{p}_{d}_{download.suggested_filename}"
                                        
                                        # บันทึกไฟล์ลงในโฟลเดอร์ที่เหมาะสม
                                        await download.save_as(f"./{parent_dir}/{a}/{p}/{new_filename}")
                                        print(f"      -> บันทึกไฟล์เรียบร้อยแล้ว: {new_filename}")
                                        await page.reload()
                                        
                                except Exception as e:
                                    print(f"เกิดข้อผิดพลาดในการดาวน์โหลดไฟล์สำหรับ {a} / {p} / {d}: {e}")
                                    broken_excel_file.append([a, p, d])
                                    await page.reload()
                                    
                        # ---------------------------------------
                        # ตรวจสอบไฟล์
                        # ---------------------------------------
                        
                        # ดึงรายชื่อไฟล์ที่ดาวน์โหลดมาได้
                        downloaded_files = os.listdir(province_dir)
                        
                        # สร้าง set ของชื่ออำเภอที่ดาวน์โหลดมาได้
                        downloaded_districts = set()
                        for filename in downloaded_files:
                            parts = filename.split('_') # แยกชื่ออำเภอจากชื่อไฟล์ที่กำหนดเอง
                            if len(parts) >= 2:
                                downloaded_districts.add(parts[1])
                                
                        # เปรียบเทียบสอง set เพื่อหาชื่ออำเภอที่ขาดหายไป
                        missing_districts = expected_districts - downloaded_districts
                        if missing_districts:
                            incomplete_downloads[p] = list(missing_districts)
                            print(f" ⚠️ ไฟล์สำหรับจังหวัด {p} ดาวน์โหลดไม่ครบ ขาด: {missing_districts}")
                            
                            # เขียนข้อมูลลงไฟล์ log
                            current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
                            with open(log_path, 'a', encoding='utf-8') as f:
                                for missing_d in missing_districts:
                                    f.write(f"{current_time} : {a}, {p}, {missing_d}\n")
                        else:
                            print(f" ✅ ไฟล์สำหรับจังหวัด {p} ดาวน์โหลดครบทุกอำเภอแล้ว")
            
        await browser.close() 

        end_time = datetime.now()
        diff_time = end_time - start_time
        print(diff_time) 
        print(f"\nการดำเนินการทั้งหมดเสร็จสิ้นแล้ว")
        print(f"ใช้เวลาไปทั้งหมด: {diff_time}")
        print(f"ไฟล์ที่ไม่สามารถดาวน์โหลดได้: {broken_excel_file}")
        print(f"ไฟล์ที่ดาวน์โหลดไม่ครบ: {incomplete_downloads}")

await main()

# Append all diw datas

In [1]:
import os
import pandas as pd
from datetime import datetime

start_time = datetime.now()

# Define the root directory to search for files
root_dir = "./diw"
child_dir = 'all'

# Output file path
output_dir = os.path.join(root_dir, child_dir)
output_filename = "diw_client.xlsx"
output_path = os.path.join(output_dir, output_filename)

# List to store dataframes
dfs = []

print("Starting to process Excel files...")

# Use os.walk to traverse the directory tree
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Exclude the output directory from being processed
    if child_dir in dirnames:
        dirnames.remove(child_dir)
    
    # Iterate through all files in the current directory
    for filename in filenames:
        # Check if the file is an Excel file and not the output file itself
        if filename.endswith(".xls") or filename.endswith(".xlsx"):
            # Construct the full file path
            file_path = os.path.join(dirpath, filename)
            
            try:
                # Read the Excel file into a DataFrame
                df = pd.read_excel(file_path)
                
                # Drop the last two rows ที่เป็นเศษทิ้ง เพราะไม่ใช้
                df = df.iloc[:-2, :]
                
                # Split the path and get the folder name that represents the region.
                # The folder list should be ['diw', 'กทม. และภาคกลาง', 'จ.กรุงเทพมหานคร', ...]
                # The region name is at index 1
                path_parts = dirpath.split(os.sep)
                # Ensure the path has at least two parts to avoid an index error
                if len(path_parts) > 1:
                    area_name = path_parts[1]
                else:
                    # Fallback to a default or the full path if the structure is unexpected
                    area_name = "Other"
                
                df['Area'] = area_name
                
                dfs.append(df)
                print(f"  - Processed: {file_path}")
            except Exception as e:
                print(f"  - ERROR processing {file_path}: {e}")

# Check if any data was found before concatenation
if not dfs:
    print("No Excel files were found to process.")
else:
    # Concatenate all dataframes into a single dataframe
    df_concat = pd.concat(dfs, axis=0)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the combined DataFrame to the specified path
    df_concat.to_excel(output_path, index=False)
    print(f"\nSuccessfully combined all data into {output_path}")

end_time = datetime.now()
diff_time = end_time - start_time
print(f"\nTotal processing time: {diff_time}")


Starting to process Excel files...
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองสาน_10-18-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองสามวา_10-46-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คลองเตย_10-33-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.คันนายาว_10-43-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.จตุจักร_10-30-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.จอมทอง_10-35-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดอนเมือง_10-36-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดินแดง_10-26-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ดุสิต_10-2-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหานคร_อ.ตลิ่งชัน_10-19-.xls
  - Processed: ./diw\กทม. และภาคกลาง\จ.กรุงเทพมหานคร\จ.กรุงเทพมหา

---